使用Python对DBMS进行操作的时候,一般经过四个步骤:
- 引入API模块
- 与数据库建立连接
- 执行SQL语句
- 关闭数据连接

使用Python对数据库进行访问需要基于DB API规范,有很多库可供选择,如MySQLLdb,mysqlclint,PyMySQL,peewee和SQLAlchemy等.下面以mysql-connector为例,这是MySQL官方提供的驱动器,用以为后端语言提供连接

# 如何使用mysql-connector

首先安装mysql-connector,使用pip install mysql-connector

In [94]:
# -*- coding:UTF-8 -*-
import mysql.connector
# 打开数据连接
db = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='5276',
    database='sql_learn',
    auth_plugin='mysql_native_password')
# 获取操作游标
cursor = db.cursor()
# 执行sql语句
cursor.execute("SELECT VERSION()")
# 获取一条数据
data = cursor.fetchone()
print("MySQL 版本:{}".format(data))
# 关闭游标&数据库连接
# cursor.close()
# db.close()

MySQL 版本:('8.0.15',)


以上代码中有两个重要的对象,分别是Connection和Cursor

Connection就是对数据库的当前连接进行管理,具体操作如下:
- 通过指定host,user,passwd和port等参数来创建数据库连接,这些参数分别对应数据库的IP地址,用户名,密码和端口号
- 使用db.close()关闭数据连接
- 使用db.cursor()创建游标,操作数据库中的数据
- 使用db.begin()开启事务
- 使用db.commit()和db.rollback(),对事务进行提交以及回滚

当通过cursor=db.cursor()创建游标后,便可以通过面向过程的编程方式对数据库的数据进行操作:
- 使用cursor.execute(query_sql),执行数据库查询
- 使用cursor.fetchone(),读取数据集中的一条数据
- 使用cursor.fetchall(),取出数据集中的所有行,返回一个元组tuples类型
- 使用cursor.fetchmany(n),取出数据集中的多条数据,同样返回一个元组tuples
- 使用cursor.rowcount,返回查询结果集中的行数.如果没有查询到数据或者还没有查询,则结果为-1,否则会返回查询得到的数据行数
- 使用cursor.close(),关闭游标

# 对数据表进行增删改查

下面来实操下如何对数据库中heros数据表进行CRUD的操作,即增加,读取,更新和删除

## 增加数据

In [95]:
# 插入新球员
sql = "INSERT INTO player (team_id,player_name,height) values (%s,%s,%s)"
val = (1003, "约翰-科林斯", 2.08)
cursor.execute(sql, val)
db.commit()
print(cursor.rowcount, "记录插入成功.")

1 记录插入成功.


sql是相应的SQL语句,val为SQL语句中参数,SQL执行后使用db.commit()进行提交(不提交的话数据不会被插入).需要说明的是,我们在使用SQL语句的时候,可以向SQL语句传递参数,这时SQL语句里要统一用(%s)进行占位,否则会报错.**不论插入的数值为整数类型还是浮点类型,都需要统一用(%s)进行占位.

## 读取数据

In [96]:
# 查询身高大于等于2.08的球员
sql = 'SELECT player_id,player_name,height FROM player WHERE height>=2.08'
cursor.execute(sql)
data = cursor.fetchall()
for each_player in data:
    print(each_player)

(10003, '安德烈-德拉蒙德', 2.11)
(10004, '索恩-马克', 2.16)
(10009, '扎扎-帕楚里亚', 2.11)
(10010, '乔恩-洛伊尔', 2.08)
(10011, '布雷克-格里芬', 2.08)
(10015, '亨利-埃伦森', 2.11)
(10023, '多曼塔斯-萨博尼斯', 2.11)
(10024, '迈尔斯-特纳', 2.11)
(10032, 'TJ-利夫', 2.08)
(10033, '凯尔-奥奎因', 2.08)
(10037, 'Ike Anigbogu', 2.08)
(10047, '约翰-柯林斯', 2.08)
(10056, '约翰-科林斯', 2.08)


## 修改数据

In [97]:
# 修改球员约翰-柯林斯
sql = 'UPDATE player SET height=%s WHERE player_name=%s'
val = (2.09, "约翰-科林斯")
cursor.execute(sql, val)
db.commit()
print(cursor.rowcount, "记录被修改.")

1 记录被修改.


## 删除数据

In [98]:
sql = 'DELETE FROM player WHERE player_name ="约翰-科林斯"'
cursor.execute(sql)
db.commit()
print(cursor.rowcount, " 记录删除成功。")

1  记录删除成功。


## 操作CRUD相关注意事项

- 打开数据库连接以后,如果不再使用要及时关闭以免造成资源浪费
- 进行CRUD操作时可能会出现异常,这时需要用try...except捕获异常信息.

In [68]:
import traceback
try:
  sql = "INSERT INTO player (team_id, player_name, height) VALUES (%s, %s, %s)"
  val = (1003, "约翰-柯林斯", 2.08)
  cursor.execute(sql, val)
  db.commit()
  print(cursor.rowcount, " 记录插入成功。")
except Exception as e:
  # 打印异常信息
  traceback.print_exc()
  # 回滚  
  db.rollback()
finally:
  # 关闭数据库连接
  db.close()

Traceback (most recent call last):
  File "<ipython-input-68-530c4bd09774>", line 5, in <module>
    cursor.execute(sql, val)
  File "D:\YY\Anaconda\envs\Analysis\lib\site-packages\mysql\connector\cursor.py", line 551, in execute
    self._handle_result(self._connection.cmd_query(stmt))
  File "D:\YY\Anaconda\envs\Analysis\lib\site-packages\mysql\connector\connection.py", line 490, in cmd_query
    result = self._handle_result(self._send_cmd(ServerCmd.QUERY, query))
  File "D:\YY\Anaconda\envs\Analysis\lib\site-packages\mysql\connector\connection.py", line 395, in _handle_result
    raise errors.get_exception(packet)
mysql.connector.errors.IntegrityError: 1062 (23000): Duplicate entry '约翰-柯林斯' for key 'player_name'


用Python操作MySQL方式很多,mysql-connector只是其中一种,实际上还有另外一种方式,就是采用ORM框架.ORM的英文是Object Relational Mapping ,也就是采用对象关系映射的模式.使用这种模式可以将数据库中各种数据表之间的关系映射到程序中的对象.这种模式屏蔽了底层的数据库细节,不需要我们与复杂的SQL语句打交道,直接采用操作对象的形式操作就可以.